# اتصال گوگل کولب با گوگل درایو

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# خواندن تصاویر

In [2]:
import cv2
import numpy as np
import glob
width=100
height=100
Data=[]
Target=[];

file_list = glob.glob(r'/content/drive/My Drive/DataSet/Covid/covid/*.png')
for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(0)

file_list = glob.glob(r'/content/drive/My Drive/DataSet/Covid/Non covid/*.png')
for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(1)
  


# آماده سازی داده

In [3]:
import keras
from keras.utils import np_utils
Instance=np.reshape(Data,(len(Data),height,width,1))
Target = np_utils.to_categorical(Target)

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(Instance,Target,test_size=0.3,random_state=42,stratify=Target, shuffle=True)

# ایجاد شبکه موازی

In [5]:
from keras.models import Model
from keras import layers
import keras

In [6]:
from keras.layers  import BatchNormalization,Dropout,MaxPool2D,Dropout,Concatenate,Input,Conv2D,Flatten,Dense
print("Creating Model...")
inputs = Input(shape=(height,width,1))

conv_0 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(inputs)
conv_0=BatchNormalization()(conv_0)
maxpool_0 = MaxPool2D(pool_size=(2,2))(conv_0)
drop_0 = Dropout(0.2)(maxpool_0)

conv_1 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(drop_0)
conv_1=BatchNormalization()(conv_1)
maxpool_1 = MaxPool2D(pool_size=(2,2))(conv_1)
drop_1 = Dropout(0.2)(maxpool_1)

conv_2 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(drop_1)
conv_2=BatchNormalization()(conv_2)
maxpool_2 = MaxPool2D(pool_size=(2,2))(conv_2)
drop_2 = Dropout(0.2)(maxpool_2)

# Network 2
conv_0_1 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(inputs)
conv_0_1 =BatchNormalization()(conv_0_1 )
maxpool_0_1 = MaxPool2D(pool_size=(2,2))(conv_0_1 )
drop_0_1 = Dropout(0.2)(maxpool_0_1)

conv_1_1 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(drop_0_1)
conv_1_1 =BatchNormalization()(conv_1_1 )
maxpool_1_1 = MaxPool2D(pool_size=(2,2))(conv_1_1 )
drop_1_1 = Dropout(0.2)(maxpool_1_1)

conv_2_1 = Conv2D(128, kernel_size=(3,3), padding='same', activation='relu')(drop_1_1)
conv_2_1=BatchNormalization()(conv_2_1)
maxpool_2_1 = MaxPool2D(pool_size=(2,2))(conv_2_1)
drop_2_1 = Dropout(0.2)(maxpool_2_1)


# Network 3

conv_0_3 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(inputs)
conv_0_3 = Conv2D(32, kernel_size=(3,3), padding='same', activation='relu')(conv_0_3)
conv_0_3=BatchNormalization()(conv_0_3)
maxpool_0_3 = MaxPool2D(pool_size=(2,2))(conv_0_3)
drop_0_3=Dropout(0.2)(maxpool_0_3)

conv_1_3 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(drop_0_3)
conv_1_3 = Conv2D(64, kernel_size=(3,3), padding='same', activation='relu')(conv_1_3)
conv_1_3=BatchNormalization()(conv_1_3)
maxpool_1_3 = MaxPool2D(pool_size=(2,2))(conv_1_3)
drop_1_3=Dropout(0.3)(maxpool_1_3)

# Merge

flatten0 = Flatten()(drop_2)
flatten1 = Flatten()(drop_2_1)
flatten2 = Flatten()(drop_1_3)
concatenated_tensor = Concatenate(axis=1)([flatten0, flatten1, flatten2])



dropout = Dropout(0.2)(concatenated_tensor)
Danse2=Dense(500)(dropout)
#Extract_model=Model(inputs=inputs, outputs=Danse2)
output = Dense(units=2, activation='softmax')(Danse2)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

#adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)


model.compile(optimizer="adam", loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
print("Traning Model...")
model.fit(X_train, y_train, batch_size=100, epochs=50, verbose=1, validation_data=(X_test, y_test))  # starts training

Creating Model...
Traning Model...
Epoch 1/50
25/25 [==============================] - 358s 14s/step - loss: 54.3043 - accuracy: 0.7447 - val_loss: 109.5191 - val_accuracy: 0.5793
Epoch 2/50
25/25 [==============================] - 359s 14s/step - loss: 8.7413 - accuracy: 0.8449 - val_loss: 29.1239 - val_accuracy: 0.6235
Epoch 3/50
25/25 [==============================] - 363s 15s/step - loss: 3.5388 - accuracy: 0.8776 - val_loss: 4.3820 - val_accuracy: 0.8141
Epoch 4/50
25/25 [==============================] - 355s 14s/step - loss: 2.0367 - accuracy: 0.9082 - val_loss: 7.1858 - val_accuracy: 0.7390
Epoch 5/50
25/25 [==============================] - 365s 15s/step - loss: 1.5944 - accuracy: 0.9227 - val_loss: 2.8596 - val_accuracy: 0.8394
Epoch 6/50
25/25 [==============================] - 359s 14s/step - loss: 1.7022 - accuracy: 0.9190 - val_loss: 4.2088 - val_accuracy: 0.7887
Epoch 7/50
25/25 [==============================] - 347s 14s/step - loss: 1.2373 - accuracy: 0.9215 - val_los

# ارزیابی